In [ ]:
!mkdir /home/aistudio/external-libraries

!pip install chardet==3.0.4  

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [ ]:
!zip -r jsdbzcm.zip /home/aistudio/

In [ ]:
!pip install pgl==1.2.0  

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 7.9MB 15.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 19.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 16.7MB/s eta 0:00:01


In [ ]:
!pip install easydict -q -t /home/aistudio/external-libraries

In [ ]:
!unzip -oq /home/aistudio/work/graph.zip -d work/

In [ ]:
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd

In [ ]:
!pip install pgl==1.2.0

In [ ]:
#!python train2.py --dataset dataset

In [ ]:
from easydict import EasyDict as edict
config = {
    "model_name": "ResGAT",
    "num_layers": 2,
    "hidden_size": 32,
    "heads":4,
    "learning_rate": 0.05,
    "dropout": 0.1,
    "weight_decay": 0.001,
    "edge_dropout": 0.1,
    
     }
config = edict(config)

In [ ]:
from collections import namedtuple

Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])

def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("work/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])
    
    return edges

In [ ]:
import numpy as np
import pandas as pd



df2 = pd.read_csv("/home/aistudio/submissionqb.csv")

node_index2 = df2["nid"].values
node_label2 = df2["label"].values
node_index2=np.zeros(node_index2.shape,dtype='float32')
node_label2=np.zeros(node_label2.shape,dtype='int')
def data_add(feat,edge,train,label):
    train0_len=len(train)

    

    train=np.concatenate([train,node_index2],axis=0)
    label=np.concatenate([label,node_label2],axis=0)

    feat_dim=feat.shape[1]
    feat_zero=np.zeros((feat.shape[0],35),dtype='float32')
    #feat=np.zeros(feat.shape)
    feat=np.concatenate([feat,feat_zero],axis=1)
    feat_len=len(feat)
    train_len=len(train)
    train_copy=[]
    label_copy=[]
    feat_copy=[]
    src=[]
    dsc=[]
    tr={}
    
    for i in range(train_len):
        
        train_copy.append(i+feat_len)
        label_copy.append(label[i])

        feat_tmp=np.zeros(feat.shape[1],dtype='float32')
        if i<train0_len:
            if np.random.random()<0.825:
                feat_tmp[feat_dim+int(label[i])]=1
            else:
                feat_tmp[feat_dim+np.random.randint(0,35)]=1
        else:
            if np.random.random()<0.75:
                feat_tmp[feat_dim+int(label[i])]=1
            else:
                feat_tmp[feat_dim+np.random.randint(0,35)]=0
        feat_copy.append(feat_tmp)
        src.append(train[i])
        dsc.append(i+feat_len)
        src.append(i+feat_len)
        dsc.append(train[i])
        src.append(i+feat_len)
        dsc.append(i+feat_len)

 
    feat_copy=np.array(feat_copy).astype('float32')
    train_copy=np.array(train_copy)
    label_copy=np.array(label_copy)
    edge_copy = np.vstack([src, dsc]).T
    edges = np.vstack([edge, edge_copy])    
    feats=np.vstack([feat,feat_copy]).astype('float32')
    return feats,edges

In [ ]:
def load():
    # 从数据中读取点特征和边，以及数据划分

    
    df = pd.read_csv("work/train.csv")
    node_index = df["nid"].values
    node_label = df["label"].values
    train_part = int(len(node_index) * 0.8)
    train_index = node_index[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_index[train_part:]
    valid_label = node_label[train_part:]
    test_index = pd.read_csv("work/test.csv")["nid"].values

    
    
    node_feat = np.load("work/feat.npy")
    test0=0
    if test0==1:
        feat_min=node_feat.min(axis=0)
        feat_max=node_feat.max(axis=0)
        feat_cs=(feat_max-feat_min)/35
        feat_n=np.random.randint(0,35,feat_min.shape)
        feat_cs=feat_cs*feat_n
        node_feat=np.mod(node_feat+feat_cs,feat_max)+np.floor_divide(node_feat+feat_cs,feat_max)*feat_min
        node_feat=node_feat.astype('float32')
        


   
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    node_feat,edges=data_add(node_feat,edges,node_index,node_label)
    
 
 
    num_nodes = node_feat.shape[0]

    
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})


    indegree = graph.indegree()

    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)



    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    
         
    return dataset

In [ ]:
import paddle.fluid as F
dataset= load()

train_index = dataset.train_index
train_index = np.expand_dims(train_index, -1)
train_label=np.reshape(dataset.train_label,[-1,1])
val_index = dataset.valid_index
val_label = np.reshape(dataset.valid_label, [-1, 1])
val_index = np.expand_dims(val_index, -1)

test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
test_label = np.zeros((len(test_index), 1), dtype="int64")




In [ ]:
import pgl
import model
import paddle.fluid as fluid
import numpy as np
import time
from build_model import build_model

# 使用CPU
#place = fluid.CPUPlace()

# 使用GPU
place = fluid.CUDAPlace(0)

train_program = fluid.default_main_program()
startup_program = fluid.default_startup_program()
with fluid.program_guard(train_program, startup_program):
    with fluid.unique_name.guard():
        gw, loss, acc, pred,pred1 = build_model(dataset,
                            config=config,
                            phase="train",
                            main_prog=train_program)

test_program = fluid.Program()
with fluid.program_guard(test_program, startup_program):
    with fluid.unique_name.guard():
        _gw, v_loss, v_acc, v_pred,v_pred1 = build_model(dataset,
            config=config,
            phase="test",
            main_prog=test_program)


test_program = test_program.clone(for_test=True)

exe = fluid.Executor(place)
startup=0

In [10]:
from visualdl import LogWriter
import os
log_writer = LogWriter(logdir="./log")

epoch =600
if startup==0:
    exe.run(startup_program)
    startup=1

# 将图数据变成 feed_dict 用于传入Paddle Excecutor
feed_dict = gw.to_feed(dataset.graph)
best_acc = 0.6
is_load=True
if is_load==True:
    if os.path.isdir('./train_model/'):
        pass
        fluid.load(test_program, os.path.join('./train_model/', 'model'), exe)



for epoch in range(epoch):
    # Full Batch 训练
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(train_index, dtype="int64")
    feed_dict["node_label"] = np.array(train_label, dtype="int64")
    




    train_loss, train_acc = exe.run(train_program,
                                feed=feed_dict,
                                fetch_list=[loss, acc],
                                return_numpy=True)

    # Full Batch 验证
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    print("Epoch", epoch, "Train Acc", train_acc[0], "Valid Acc", val_acc[0],"Valid Loss", val_loss[0])
    if epoch%1==0:
        #print("Epoch", epoch, "iter", iter,"Train Acc", train_acc[0], "Valid Acc", val_acc[0],"train loss",train_loss,"valid loss",val_loss)
        log_writer.add_scalar(tag = 'train_acc', step = epoch, value = train_acc)
        log_writer.add_scalar(tag = 'val_acc', step = epoch, value = val_acc)
        log_writer.add_scalar(tag = 'train_loss', step = epoch, value =train_loss)
        log_writer.add_scalar(tag = 'val_loss', step = epoch, value = val_loss)

    #if val_acc[0]>0.75:
    #    break

    if  val_acc[0]> best_acc:
        best_acc = val_acc[0]
        ckpt_dir =  os.path.join('./', 'best_model')
        print("Save model checkpoint to {}".format(ckpt_dir))
        if not os.path.isdir(ckpt_dir):
            os.makedirs(ckpt_dir)
        fluid.save(test_program, os.path.join(ckpt_dir, 'model')) 

        ckpt_dir2 =  os.path.join('./', 'train_model')
        print("Save model checkpoint to {}".format(ckpt_dir2))
        if not os.path.isdir(ckpt_dir2):
            os.makedirs(ckpt_dir2)
        fluid.save(train_program, os.path.join(ckpt_dir2, 'model'))    

Epoch 0 Train Acc 0.7361714 Valid Acc 0.8653805 Valid Loss 0.46011794
Save model checkpoint to ./best_model
Save model checkpoint to ./train_model
Epoch 1 Train Acc 0.7190681 Valid Acc 0.85968536 Valid Loss 0.47810963
Epoch 2 Train Acc 0.7056133 Valid Acc 0.87805223 Valid Loss 0.42883128
Save model checkpoint to ./best_model
Save model checkpoint to ./train_model
Epoch 3 Train Acc 0.72985333 Valid Acc 0.8562682 Valid Loss 0.49697655
Epoch 4 Train Acc 0.71073896 Valid Acc 0.8628889 Valid Loss 0.47042045
Epoch 5 Train Acc 0.7159892 Valid Acc 0.88438815 Valid Loss 0.4119956
Save model checkpoint to ./best_model
Save model checkpoint to ./train_model
Epoch 6 Train Acc 0.7287855 Valid Acc 0.8713604 Valid Loss 0.44283432
Epoch 7 Train Acc 0.7145298 Valid Acc 0.8709333 Valid Loss 0.44836152
Epoch 8 Train Acc 0.720136 Valid Acc 0.87968963 Valid Loss 0.43042922
Epoch 9 Train Acc 0.7251015 Valid Acc 0.88025916 Valid Loss 0.42131746
Epoch 10 Train Acc 0.7302449 Valid Acc 0.8831067 Valid Loss 0.41

In [11]:

feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签


if os.path.isdir('./best_model/'):
    fluid.load(test_program, os.path.join('./best_model/', 'model'), exe)
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]

In [12]:
submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission0.csv", index=False)

In [ ]:
qb_index=[]
qb_label=[]
for i in range(130644):
    qb_index.append(i)
    qb_label.append(0)

qb_index=np.array(qb_index).reshape(-1,1)
qb_label=np.array(qb_label).reshape(-1,1)
feed_dict["node_index"] = np.array(qb_index, dtype="int64")
feed_dict["node_label"] = np.array(qb_label, dtype="int64") #假标签

if os.path.isdir('./best_model/'):
    fluid.load(test_program, os.path.join('./best_model/', 'model'), exe)
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]
submission = pd.DataFrame(data={
                            "nid": qb_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submissionqb.csv", index=False)

In [14]:
def publicnum(num, d = 0):
    dictnum = {}
    for i in range(len(num)):
        if num[i] in dictnum.keys():
            dictnum[num[i]] += 1
        else:
            dictnum.setdefault(num[i], 1)
    maxnum = 0
    maxkey = 0
    for k, v in dictnum.items():
        if v > maxnum:
            maxnum = v
            maxkey = k
    return maxkey

df0=pd.read_csv("submission0.csv")
df1=pd.read_csv("submission1.csv")
df2=pd.read_csv("submission2.csv")
df3=pd.read_csv("submission3.csv")
df4=pd.read_csv("submission4.csv")
df5=pd.read_csv("submission5.csv")
df6=pd.read_csv("submission6.csv")
df7=pd.read_csv("submission7.csv")
#df8=pd.read_csv("submission8.csv")
#df9=pd.read_csv("submission9.csv")
#df10=pd.read_csv("submission10.csv")




nids=[]
labels=[]

for i in range(df4.shape[0]):
    label_zs=[]
    label_zs.append(df0.label[i])
    label_zs.append(df1.label[i])
    label_zs.append(df2.label[i])
    label_zs.append(df3.label[i])
    label_zs.append(df4.label[i])
    label_zs.append(df5.label[i])
    label_zs.append(df6.label[i])
    label_zs.append(df7.label[i])
    #label_zs.append(df8.label[i])
    #label_zs.append(df9.label[i])
    #label_zs.append(df10.label[i])
    lab=publicnum(label_zs, d = 0)
    labels.append(lab)
    nids.append(df4.nid[i])


submission = pd.DataFrame(data={
                            "nid": nids,
                            "label": labels
                        })
submission.to_csv("submissiona.csv", index=False)